<a href="https://colab.research.google.com/github/robertheubanks/AI-Engineering-Bootcamp-Homework/blob/main/Eubanks_Week2_Assignment_4_OpenAI_Assistants_Building_Agentic_RAG_with_Function_Calling_API_and_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [1]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


In [2]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [3]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [4]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "pharmaceutical medical affairs" # @param {type: "string"}
instructions = "Integrate the intended audience in the prompt, e.g., the audience is an expert in the field." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [5]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [6]:
assistant

Assistant(id='asst_RqFcNcV7q5Tgpe1g1KntLqKy', created_at=1708808796, description=None, file_ids=[], instructions='Integrate the intended audience in the prompt, e.g., the audience is an expert in the field.', metadata={}, model='gpt-3.5-turbo', name='pharmaceutical medical affairs', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [7]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [8]:
thread

Thread(id='thread_rU95OKwF9hkd4AatHg8nCzPR', created_at=1708808819, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [10]:
message

ThreadMessage(id='msg_n9uLZk5FvAVGzsx1HzBFo9v9', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1708808855, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_rU95OKwF9hkd4AatHg8nCzPR')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [15]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Break down complex tasks into a sequence of simpler prompts in an interactive conversation." # @param {type: "string"}

Let's run our Thread!

In [16]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [17]:
run

Run(id='run_4lzU51Q647ocgIb8cNeNPsmz', assistant_id='asst_RqFcNcV7q5Tgpe1g1KntLqKy', cancelled_at=None, completed_at=None, created_at=1708809835, expires_at=1708810435, failed_at=None, file_ids=[], instructions='Integrate the intended audience in the prompt, e.g., the audience is an expert in the field. Break down complex tasks into a sequence of simpler prompts in an interactive conversation.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_rU95OKwF9hkd4AatHg8nCzPR', tools=[], usage=None)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [18]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [19]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [20]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [21]:
messages.data[0]

ThreadMessage(id='msg_o6ZBWXXEauADWI0Hox26M1Ji', assistant_id='asst_RqFcNcV7q5Tgpe1g1KntLqKy', content=[MessageContentText(text=Text(annotations=[], value='What specific information or assistance do you need help with? Feel free to ask any questions or provide details about the task you would like assistance with.'), type='text')], created_at=1708809836, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_4lzU51Q647ocgIb8cNeNPsmz', thread_id='thread_rU95OKwF9hkd4AatHg8nCzPR')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [22]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [23]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [24]:
file_reference

FileObject(id='file-beVB3N0ACDPUuWWbRQoOg7cn', bytes=1199, created_at=1708810429, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [25]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [26]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


Let's look at the final result!

In [27]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_FOd9fd9LfNP9jxg61B7CZpq8', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the first words Victor Frankenstein speaks?'), type='text')], created_at=1708810459, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_2C7lu8j84vou5AuzN4gHCgly')], object='list', first_id='msg_FOd9fd9LfNP9jxg61B7CZpq8', last_id='msg_FOd9fd9LfNP9jxg61B7CZpq8', has_more=False)

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [28]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [29]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [30]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [31]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [32]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [33]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_iE8dKyR7JoDWtIDHQK3JNDMp'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_gPJfsOpVcFjgv9Xl5uLI0cZm', code_interpreter=CodeInterpreter(input="# Read the first few lines of the file to identify its content\r\nwith open(file_path, 'r') as file:\r\n    first_few_lines = [next(file) for _ in range(5)]\r\n\r\nfirst_few_lines", outputs=[CodeInterpreterOutputLogs(logs="['--2024-02-24 21:33:36--  https://www.gutenberg.org/files/84/84-h/84-h.htm\\n',\n 'Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47\\n',\n 'Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.\\n',\n 'HTTP request sent, awaiting response... 200 OK\\n',\n 'Length: 466919 (456K) [text/html]\\n']", type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_kmYQ5hbN7

In [34]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_iE8dKyR7JoDWtIDHQK3JNDMp', assistant_id='asst_UPFWO69Kk8CNDGlZh7zJ2vc1', content=[MessageContentText(text=Text(annotations=[], value='The first few lines of the file contain information about a web request to "https://www.gutenberg.org/files/84/84-h/84-h.htm". It appears to be an HTML file from Project Gutenberg (a digital library of free eBooks). \n\nTherefore, based on the content, it is identified as an HTML file. Is there anything else you would like to know or do with this file?'), type='text')], created_at=1708810542, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_oQvKAvCjMmXMAiGnsheuJePR', thread_id='thread_rmfp7YdgGPYRYo6YcJ7gWyLa'), ThreadMessage(id='msg_kmYQ5hbN7vFyzhdAsmQediA3', assistant_id='asst_UPFWO69Kk8CNDGlZh7zJ2vc1', content=[MessageContentText(text=Text(annotations=[], value='The file type could not be determined based on the file extension. Let me try to read the file to see 

In [35]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

NotFoundError: Error code: 404 - {'error': {'message': "No file found with id 'file-beVB3N0ACDPUuWWbRQoOg7cn'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [36]:
!pip install -qU duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 110.2 MB/s eta 0:00:00


In [37]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [38]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

'NHL.com The official 2023 - 2024 roster of the Winnipeg Jets, including position, height, weight, date of birth, age, and birth place.\nManitoba Adam Lowry named new captain of Winnipeg Jets 30-year-old centre is 3rd captain since team relocated to Winnipeg from Atlanta Darren Bernhardt · CBC News · Posted: Sep 12, 2023...\nWinnipeg Jets general manager Kevin Cheveldayoff announced Adam Lowry as the new team captain of the Winnipeg Jets on tuesday. Lowry will follow Andrew Ladd and Blake Wheeler to serve as...\nMost Goals, Season: Ilya Kovalchuk (2005-06), Ilya Kovalchuk (2007-08), 52 Most Points, Season: Marián Hossa (2006-07), 100 Become a Stathead & surf this site ad-free. Jets History Leaders Skaters Goalies Coaches Draft Captains Sweater #s All-Time W-L Records More Captains More Jets Pages Jets History More Jets Pages Leaders Season Leaders\nTrade deadline Jets name Adam Lowry as captain ahead of 2023-24 season: Why Winnipeg chose him By Murat Ates and The Athletic Staff Sep 12,

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [39]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

ANSWER:
The description key-value pair in the context of defining a function for use with an API, especially one designed to be part of an assistant or AI-driven service, plays several crucial roles that are important both technically and from a user-experience perspective.


*   User Understanding: It provides a clear, concise description of what the function does, which is essential for users who may be integrating or calling this function within their own systems or workflows. This is particularly valuable in environments where developers may be dealing with a large number of available functions and need to quickly understand the purpose and utility of each one.
*   Documentation: Automatically generated documentation can leverage these descriptions to provide more insightful and useful information. This improves the overall documentation quality and usability for developers who are exploring or learning about the API.
*   Searchability: In a large API with many endpoints or functions, the description helps in indexing the functions for search operations. Developers can find the functions they need by searching for keywords or phrases, making the API more accessible and easier to use.
*   Integrations and Compatibility: For systems that may dynamically call APIs based on certain criteria or inputs, the description provides a human-readable explanation of the function's capabilities, aiding in the selection process for automated workflows or AI-driven decision-making processes.
*   Accessibility: In the context of making technology accessible to a broader audience, including those who may not be deeply technical, descriptions can demystify what a function does and how it can be used, thereby lowering the barrier to entry.
*   Quality Assurance and Maintenance: Descriptions also serve an internal purpose, helping developers and maintainers of the API understand the intended use and scope of each function. This can be invaluable during maintenance, debugging, and when extending the functionality of the API.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [40]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [41]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

ANSWER:
The use_assistant helper function orchestrates the interaction between a user and an AI Assistant that has the capability to call external functions as part of its processing. The function achieves this through a series of steps, structured to ensure that the assistant can not only generate responses but also execute defined functions (like duckduckgo_search) to enrich those responses with external data or computations. Here's a breakdown of its operation:

*   Thread Creation: It starts by creating a new conversation thread with the assistant. This thread acts as a container for the interaction, allowing for an organized exchange of messages between the user and the assistant.
Sending a Query: The user's query is sent to the assistant as a message within the created thread. This is the question or request the user wants the assistant to process.
*   Initiating Assistant Processing: The function then requests the assistant to start processing the user's query within the thread. This is akin to "asking" the assistant to think about the question and come up with a response.
*   Monitoring for Completion: The function enters a wait state, periodically checking if the assistant has completed processing the query. This step is crucial because the assistant might need time to consider the query, especially if external functions need to be called.
*   Handling Required Actions: If the assistant's processing requires calling an external function (like the DuckDuckGo search), the function identifies this need and executes the appropriate external function to obtain the necessary data or results.
*   Submitting External Function Outputs: Once the external function's output is obtained, it is submitted back to the assistant. This allows the assistant to incorporate the external data into its final response.
*   Finalizing the Response: The function waits again, if necessary, for the assistant to finalize its response, now with the added context or data from the external function.
*   Displaying the Assistant's Messages: Finally, the function retrieves and displays the messages from the assistant found in the thread. This includes the assistant's response to the initial query, enriched or informed by the data from the external function call.

In [42]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_lREXZyaoKgkiRa5nMUtMKt3O', 'output': "NHL.com The official 2023 - 2024 roster of the Winnipeg Jets, including position, height, weight, date of birth, age, and birth place.\nWinnipeg Jets general manager Kevin Cheveldayoff announced Adam Lowry as the new team captain of the Winnipeg Jets on tuesday. Lowry will follow Andrew Ladd and Blake Wheeler to serve as...\nSault Ste. Marie Toronto Vancouver Vancouver Island Windsor Winnipeg Advertisement Winnipeg News Adam Lowry named as captain of the Winnipeg Jets Winnipeg Jets' Adam Lowry celebrates...\nManitoba Adam Lowry named new captain of Winnipeg Jets 30-year-old centre is 3rd captain since team relocated to Winnipeg from Atlanta Darren Bernhardt · CBC News · Posted: Sep 12, 2023...\nBy Jamie Thomas @JamieThomasTV WinnipegJets.com September 12, 2023 There are not many honours in team sports bigger t

'thread_4Xdxmp8IovHfd4mjA8eU14GR'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [43]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [44]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The uploaded file does not contain information about the current Captain of the Winnipeg Jets. Would you like me to search for this information online?
user: Who is the current Captain of the Winnipeg Jets?


'thread_Q65M42M8Ywzoarpj9xHjcyIB'

In [45]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is Mary Shelley.
user: Who is the author of the supplied file?


'thread_GX2AsMoYjp7efNWTPGYmxee1'

In [46]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The provided file is 466,919 bytes in size.
user: How many bytes is the provided file?


'thread_HNmL9yjmZRL0haVFzGBO2ggr'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

ANSWER:
In the context provided, where an Assistant is equipped with various tools (such as a code interpreter, retrieval tool, and a custom function like duckduckgo_search), the decision-making process regarding which tool to use for a given query is guided by the Assistant's underlying AI model and its instructions. Here’s how it works:
*   Understanding the Query: When a query is received, the Assistant's AI model analyzes the content and context of the query to understand its intent. This involves natural language processing (NLP) capabilities to discern what the user is asking and what kind of response or action is expected.
*   Instructions Interpretation: The Assistant relies on the instructions provided during its creation to guide its response behavior. These instructions typically outline how the Assistant should handle various types of queries, potentially indicating when to use specific tools or how to prioritize them.
*   Tool Suitability Evaluation: Based on the query’s understood intent and the instructions, the Assistant evaluates which tool(s) in its arsenal are most suitable for addressing the query. This involves a form of decision-making logic that considers factors like the nature of the query (e.g., factual, computational, data retrieval) and the capabilities of each tool.
> For factual questions like "Who is the current Captain of the Winnipeg Jets?", the Assistant might decide to use the retrieval tool or the duckduckgo_search function, as these are designed to fetch real-time information from external sources.
> For queries asking about content or data within a specific file, like "Who is the author of the supplied file?" or "How many bytes is the provided file?", the Assistant might leverage the file_ids associated with it and possibly the retrieval tool to extract and compute the requested information.
*   Execution and Integration: Once the Assistant decides on the most appropriate tool(s), it executes the necessary tool(s) to gather information or perform computations. The results are then integrated into the Assistant's response generation process, ensuring that the reply is informed by the most relevant and accurate data available.
*   Dynamic Response Generation: The Assistant dynamically generates a response to the query, incorporating the outcomes from the used tool(s). This response is crafted to be coherent and directly address the user's query, based on the combination of the AI model's language capabilities and the specific information or computation provided by the tools.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [47]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [48]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_zHieFhuqsRD67u4g2064WJk6', 'output': 'How do I get the size of a file in Python? python file Share Improve this question Follow edited Apr 17, 2022 at 2:14 Mateen Ulhaq 25.5k 20 105 141 asked Jan 20, 2010 at 18:58 5YrsLaterDBA 33.9k 44 139 211 Add a comment 11 Answers Sorted by: 1484 Use os.path.getsize: >>> import os >>> os.path.getsize("/path/to/file.mp3") 2071611\nMethod 1: Using getsize function of os.path module This function takes a file path as an argument and it returns the file size (bytes). Example: Python3 # approach 1 import os file_size = os.path.getsize (\'d:/file.jpg\') print("File Size is :", file_size, "bytes") Output: File Size is : 218 bytes Method 2: Using stat function of the OS module\nUse the os.path.getsize (\'file_path\') function to check the file size. Pass the file name or file path to this function as an argument. This

Now we can observe JSON mode in action!

In [49]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [50]:
completed_flag

{'completed': False}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [ ]:
### YOUR CODE HERE
def query_until_complete(query, assistant_id):
    # Initialize the loop condition
    completed = False

    # Loop until the response is marked as complete
    while not completed:
        # Query the Assistant
        thread_id_for_response = use_assistant(query, assistant_id)

        # Fetch messages from the Assistant thread
        messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
        response = messages.data[0].content[0].text.value

        # Check if the response is complete
        completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
        completed = completed_flag['completed']

        # If not completed, possibly modify the query or handle accordingly before the next iteration
        if not completed:
            print("Response not complete. Querying again...")

            # Here, we could modify the query based on the response or just retry with the same query.
            # For simplicity, this example retries with the same query.

    # Once complete, print or return the final response
    print("Final response:", response)

# Example usage
query = "How many bytes is the provided file?"
assistant_id = assistant.id  # Assuming 'assistant' is already defined
query_until_complete(query, assistant_id)

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [51]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)